### Bayesian Network Inference Library

We will be using the pomegranate library for Bayes Net inference

  * Installation instructions https://pomegranate.readthedocs.io/en/latest/install.html
  * Tutorial / documentation https://pomegranate.readthedocs.io/en/latest/BayesianNetwork.html
  
In the tutorial / documentation, ignore the parts about "initializing a Bayesian network based completely on data" and the sections on "Probability" "Prediction" and "Fitting" -- see the example below on how to determine the probability distribution on a node in the graph based on evidence.

Just to make sure things are working, first load in the Monty Hall code from the tutorial and answer the question about whether or not a contestant should take Monty up on his offer to switch doors.

In [ ]:
from pomegranate import *

guest = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
prize = DiscreteDistribution({'A': 1./3, 'B': 1./3, 'C': 1./3})
monty = ConditionalProbabilityTable(
        [['A', 'A', 'A', 0.0],
         ['A', 'A', 'B', 0.5],
         ['A', 'A', 'C', 0.5],
         ['A', 'B', 'A', 0.0],
         ['A', 'B', 'B', 0.0],
         ['A', 'B', 'C', 1.0],
         ['A', 'C', 'A', 0.0],
         ['A', 'C', 'B', 1.0],
         ['A', 'C', 'C', 0.0],
         ['B', 'A', 'A', 0.0],
         ['B', 'A', 'B', 0.0],
         ['B', 'A', 'C', 1.0],
         ['B', 'B', 'A', 0.5],
         ['B', 'B', 'B', 0.0],
         ['B', 'B', 'C', 0.5],
         ['B', 'C', 'A', 1.0],
         ['B', 'C', 'B', 0.0],
         ['B', 'C', 'C', 0.0],
         ['C', 'A', 'A', 0.0],
         ['C', 'A', 'B', 1.0],
         ['C', 'A', 'C', 0.0],
         ['C', 'B', 'A', 1.0],
         ['C', 'B', 'B', 0.0],
         ['C', 'B', 'C', 0.0],
         ['C', 'C', 'A', 0.5],
         ['C', 'C', 'B', 0.5],
         ['C', 'C', 'C', 0.0]], [guest, prize])

s1 = Node(guest, name="guest")
s2 = Node(prize, name="prize")
s3 = Node(monty, name="monty")

model = BayesianNetwork("Monty Hall Problem")
model.add_states(s1, s2, s3)
model.add_edge(s1, s3)
model.add_edge(s2, s3)
model.bake()

In [ ]:
##  Suppose the guest chooses A, and Monty chooses B.
##  Monty gives the guest to switch from A to C.  Should she?
model.predict_proba({"guest": 'A', "monty": 'B'})

### Second Example:  Typical Noisy Sensor

* The variable **water** says whether or not there is water in my basement.  This variable takes values **{none, some, lots}**
* I have a water detector **waterDetector** that is either **on** or **off**
  * It is supposed to be **on** if and only if **water** is either some or lots
  * However, it sometimes fails to alert (is **off** when **water** is either **some** or **lots**)
  * It also sometimes false alarms (is **on** when **water** is **none**)

This is what I discovered by observing the basement over time
* On any given day, the probability of **water** is **(.98, .015, .05)** for values **(none, some, lots)**
* The likelihood of a false alarm **P(waterDetector = on | water = none) = 0.01**
* The likelihood of the sensor missing water depends on the water level: **P(waterDetector = off | water = some) = .10**;   **P(waterDetector = off | water = lots) = .005**


In [ ]:
from pomegranate import *

# This is my water node
wdist = DiscreteDistribution({"none": 0.98, "some": 0.015, "lots": 0.005})
water = Node(wdist, name="water")

# My water detector node
wddist = ConditionalProbabilityTable([["none", "on", 0.01],  ["none", "off", 0.99],
                                      ["some", "on", 0.90],  ["some", "off", 0.10],
                                      ["lots", "on", 0.995], ["lots", "off", 0.005]],
                                     [wdist])
waterDetector = Node(wddist, name="waterDetector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(water, waterDetector)
model.add_edge(water, waterDetector)
model.bake()
                                     

With no further information, what is the likelihood that (a) there is some or lots of water in my basement, and (b) what is the likelihood that my water detector is displaying ON

In [ ]:
# Compute probabilities on the basis of no additional evidence.  Its output is a list of 
# distributions over node values, in the order they were added -- in our case, water is at [0] and waterDetector is at [1]
model.predict_proba({})

In [ ]:
# This is the distribution over values of water -- not surprising, it is the same as the priors 
#  (subject to rounding error)
model.predict_proba({})[0].parameters

In [ ]:
# This is the distribution over values of waterDetector
model.predict_proba({})[1].parameters

Suppose I learn that the water detector is **on**.  How does that affect my beliefs over the basement water level

Suppose instead I go to the basement and observe that there is no water in the basement.  
How does that affect my belief as to whether or not the water detector is on?

####  Question 1 ####
My beliefs about water level change with the season.   There are two seasons, the dry season and the wet season.
The season affects my prior belief in the water level, not the behavior of the sensor.   If the season is 
**wet**  my prior distribution on **water** is
{"none": 0.80, "some": 0.15, "lots": 0.05})
and if the season is **dry** the distribution is 
{"none": 0.95, "some": 0.035, "lots": 0.015})

Adjust the model accordingly.  Suppose it's the dry season, but my water detector is saying **on** -- what do I believe about water in the basement?


In [ ]:
# This is my season node
sdist = DiscreteDistribution({"dry": 0.50, "wet": 0.50})
season = Node(sdist, name="season")

# This is my water node
wdist = ConditionalProbabilityTable([["dry", "none", 0.95],  ["dry", "some", 0.035], ["dry", "lots", 0.015],
                                     ["wet", "none", 0.80],  ["wet", "some", 0.15], ["wet", "lots", 0.05]], [sdist])
water = Node(wdist, name="water")

# My water detector node
wddist = ConditionalProbabilityTable([["none", "on", 0.01],  ["none", "off", 0.99],
                                      ["some", "on", 0.90],  ["some", "off", 0.10],
                                      ["lots", "on", 0.995], ["lots", "off", 0.005]],
                                     [wdist])
waterDetector = Node(wddist, name="waterDetector")

# The Network
model = BayesianNetwork("Water Detector")
model.add_states(season, water, waterDetector)
model.add_edge(season, water)
model.add_edge(water, waterDetector)
model.bake()

In [ ]:
model.predict_proba({"season": "dry", "waterDetector": "on"})

#### Question 2 ####
This is a very common example for Bayes Nets -- it's a variation on the example in Chapter 14 in the book.

I have an alarm in my house that can be triggered either by Burglaries or by Earthquakes, or both.
It has a false positive rate of 0.001  (that is, .1% of the time it will go off even if there is no burglary and no earthquake).

It almost always catches Burglaries:  if there is a burglary, then 99% of the time it will sound, but 1% of the time it will not.

For Earthquakes, 60% of the time it will sound, but 40% of the time it will not.  

It has never been tested in the case that there is a burglary and an earthquake simultaneously, but the company says that in that case it would sound 99.9% of the time.

I have two neighbors, John and Mary, who have promised to call me if they hear the alarm.

Mary lives close by, so if she is at home and the alarm goes off, she will report it;  also she would almost never (.01% of the time) report a false alarm.  On the other hand, she is not at home 25% of the time.

John is always at home, and so if the alarm goes off, he will almost always (90% of the time) report it.  On the other hand, he tends to get over-anxious and call me by mistake sometimes (5% of the time).

The likelihood of an earthquake is 0.002;  the likelihood of a burglary is 0.001.  There is no correlation between earthquakes and burglaries -- the fact that there is an earthquake does not make a burglary more likely, and vice versa.


Code this as a Bayes network, and answer these two questions:

1.  My phone is off so I don't know whether or not I got a call.  What is the likelihood there was a burglary?  What is the likelihood my alarm went off?

2.  I just got a call from Mary but not John.  How does that affect my belief that there was a burglary?

3.  I heard a reliable news report that there was an earthquake.  What is the chance I will will be getting a call from John soon?
